In [6]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import sys

In [7]:
from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

ImportError: cannot import name np_utils

In [46]:
def time_shift_data(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [53]:
input_1 = "pollution.csv"

dataset = read_csv(input_1,header=0,index_col=0)
values = dataset.values

input_2 = "4"

categoricalFeatA=[int(x) for x in input_2.split(";")] # "3;5" => 0 based


for n in categoricalFeatA:
    encoder = LabelEncoder()
    values[:,n] = encoder.fit_transform(values[:,n])

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

input_3 = 1

# number of lag hours (=past N hours)
n_hours = int(input_3)
n_features = len(dataset.columns)
last_feats=n_features-1

In [68]:
shifted = time_shift_data(scaled,n_hours,1)

values = shifted.values
n_train_hours = 365 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

print values

[[ 0.12977867  0.35294122  0.24590163 ...,  0.00381099  0.          0.        ]
 [ 0.14889336  0.36764708  0.24590163 ...,  0.00533197  0.          0.        ]
 [ 0.15995975  0.42647061  0.22950819 ...,  0.00839101  0.03703704  0.        ]
 ..., 
 [ 0.01006036  0.2647059   0.26229507 ...,  0.41399646  0.          0.        ]
 [ 0.01006036  0.2647059   0.26229507 ...,  0.42086649  0.          0.        ]
 [ 0.00804829  0.2647059   0.24590163 ...,  0.42621556  0.          0.        ]]
